# Loading libraries

In [23]:
import scanpy as sc
import decoupler as dc
import plotnine as p9
import liana as li
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from liana.method import MistyData, genericMistyData, lrMistyData
from liana.method.sp import RandomForestModel, LinearModel, RobustLinearModel
import re
import os
import sys

In [ ]:
!pip show anndata

# Main part

In [ ]:
input_folder_path = "/path/to/file/"

cell_grouping = "SingleR_HCA"

metadata_file_pattern = re.compile(r'filename(\d+)\.csv') # needs to be a csv file
gene_expression_pattern = re.compile(r'filename(\d+)\.csv') # needs to be a csv file
metadata_file_start = "filename_start"
gene_expression_start = "filename_start"

# get the patient (sample) ids by looking for the files
ids = []

for filename in os.listdir(input_folder_path):
    match = metadata_file_pattern.search(filename)
    if match:
        ids.append(match.group(1))
        
ids = sorted(ids, key=int)

for patient in ids[9:]:
    cell_metadata = pd.read_csv(input_folder_path + metadata_file_start + patient + ".csv")
    gene_expression_matrix = pd.read_csv(input_folder_path + gene_expression_start + patient + ".csv")
    
    # creating the anndata object
    adata = ad.AnnData(X=gene_expression_matrix.iloc[:, 1:].values, obs=cell_metadata)

    # Add spatial coordinates to the AnnData object
    spatial_coordinates = cell_metadata[['center_x', 'center_y']].values
    adata.obsm['spatial'] = spatial_coordinates
    
    gene_names = gene_expression_matrix.columns[1:]
    cell_ids = gene_expression_matrix.iloc[:, 0]
    adata.var_names = gene_names
    adata.obs_names = [str(x) for x in cell_ids]
    
    # normalize
    adata.layers['counts'] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    
    # funcomics
    # progeny = dc.get_progeny(organism='human', top=500)
    # load from progeny from file instead
    progeny = pd.read_csv('human_progeny.csv')
    
    dc.run_mlm(
        mat=adata,
        net=progeny,
        source='source',
        target='target',
        weight='weight',
        verbose=True,
        use_raw=False,
    )
    
    # extract progeny activities as an AnnData object
    acts_progeny = li.ut.obsm_to_adata(adata, 'mlm_estimate')
    
    # Get unique cell grouping labels
    cell_types = adata.obs[cell_grouping].unique()

    # Initialize the matrix with zeros
    cell_type_matrix = pd.DataFrame(0, index=adata.obs.index, columns=cell_types)

    # Set the appropriate cells to 1
    for cell in adata.obs.index:
        cell_type = adata.obs.loc[cell, cell_grouping]
        cell_type_matrix.loc[cell, cell_type] = 1

    # Add the matrix to obsm
    adata.obsm[cell_grouping] = cell_type_matrix
    
    comps = li.ut.obsm_to_adata(adata, cell_grouping)
    
    # running MISTy
    misty = genericMistyData(intra=comps, extra=acts_progeny, cutoff=0.05, bandwidth=200, n_neighs=6)
    
    misty(model=RandomForestModel, n_jobs=-1, verbose = True)
    
    interactions_df = misty.uns['interactions']
    interactions_df.to_csv('./tables/10p_interactions_' + patient + '_' + cell_grouping + '.csv', index=False)
    
    # ligand receptor
    # just this part is used in thesis
    
    sc.pp.highly_variable_genes(adata)
    hvg = adata.var[adata.var['highly_variable']].index
    
    hvg = adata.var.index
    
    misty = lrMistyData(adata[:, hvg[0:len(hvg)]], bandwidth=200, set_diag=False, cutoff=0, nz_threshold=0)
    
    misty(bypass_intra=True, model=LinearModel, verbose=True)
    
    interactions_df = misty.uns["interactions"]
    interactions_df.to_csv('./tables/10p_lr_interactions_' + patient + '_' + cell_grouping + '.csv', index=False)

In [ ]:
genes = gene_expression_matrix.columns[1:]
# resource used in lr analysis
resource = li.resource.select_resource('consensus')

# get all rows in resources where the elements in ligand and receptor column are included in the genes
filtered_resource = resource[(resource['ligand'].isin(genes)) & (resource['receptor'].isin(genes))]

pd.set_option('display.max_rows', 100)
filtered_resource